#### Load Folder

In [0]:
%run "../includes/utils"

In [0]:
from pyspark.sql.types import StringType, IntegerType, TimestampType
from pyspark.sql.functions import col, lit, current_timestamp

In [0]:
df = spark.read.format("delta").load("/Volumes/workspace/default/raw/workflows_run/")

In [0]:
df_selected_cols = df.select(col('FOLDER_ID'), col('FOLDER_NAME')).distinct()

In [0]:
df_insert_date = add_insert_date(df_selected_cols)

In [0]:
df_final = df_insert_date.withColumn('updated_date', lit(None))

In [0]:
display(df_insert_date)

#### Lendo os dados do destino, como uma tabela delta.

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("processed.folder"):
    deltaTable = DeltaTable.forName(
        spark,
        "processed.folder"
    )

    deltaTable.alias("tgt").merge(
        df_final.alias("src"),
        "tgt.FOLDER_ID = src.FOLDER_ID"
    ).whenMatchedUpdate(
        condition="tgt.FOLDER_NAME <> src.FOLDER_NAME",
        set={
            "FOLDER_NAME": "src.FOLDER_NAME",
            "updated_dt": "current_timestamp()"
        }
    ).whenNotMatchedInsert(
        values = {
            "FOLDER_ID" : "src.FOLDER_ID",
            "FOLDER_NAME" : "src.FOLDER_NAME",
            "created_dt": "src.created_date",
            "updated_dt": lit(None)
        }
    ) \
        .execute()
    #print("Table exists")
else:
    df_final.write.mode("overwrite").insertInto("processed.folder")

In [0]:
%sql
select * from processed.folder;

In [0]:
%sql
--delete from processed.folder where folder_id = 35
--update processed.folder set FOLDER_NAME = 'MDU-C' where folder_id = 76
